In [1]:
! pip install langchain
! pip install langchain-community
! pip install langchain-google-genai

# **TESTING- Simple CHATBOT**

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage , SystemMessage
from google.colab import userdata

chat_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',
                                    temperature=0,
                                    google_api_key=userdata.get('smart_IDP'))

messages = [
    SystemMessage("You are an assistant knowledgable about Healthcare. Only answer Healthcare-related questions."),
    HumanMessage(content="What is Pulmonary Artery?")
    ]

result = chat_model.invoke(messages)
print(result.content)


The **pulmonary artery** is a major blood vessel that carries deoxygenated blood from the right ventricle of the heart to the lungs.

Here are its key characteristics:

*   **Origin:** It originates from the right ventricle of the heart.
*   **Branching:** It quickly divides into the left and right pulmonary arteries, each going to one lung.
*   **Function:** Its primary role is to transport blood that is low in oxygen and high in carbon dioxide to the lungs. In the lungs, this blood releases carbon dioxide and picks up oxygen.
*   **Unique Feature:** It is the **only artery** in the adult human body that carries deoxygenated (venous) blood. All other arteries carry oxygenated blood away from the heart to the rest of the body.


# **TESTING-Using Chains and PromptTemplates**

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata
from langchain_core.output_parsers import StrOutputParser

chat_model = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    google_api_key=userdata.get('smart_IDP')
)

instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

Context: {context}

Question: {question}
"""

chat_template = ChatPromptTemplate.from_template(instruction_str)

context = "Discharge process was seamless!"
question = "Did anyone have a positive experience?"

chain = chat_template | chat_model | StrOutputParser()

result = chain.invoke({
    "context":context,
    "question":question
})

print(result)

Yes, someone had a positive experience with the discharge process, describing it as "seamless." This indicates a smooth and easy experience without any issues.


# **Using PromptTemplates & MessageTemplates**

In [4]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (ChatPromptTemplate , SystemMessagePromptTemplate , HumanMessagePromptTemplate , PromptTemplate)

chat_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash',
                                    temperature=0,
                                    google_api_key=userdata.get('smart_IDP'))

instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

Context: {context}
"""

chat_system_prompt_template = SystemMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["context"] , template=instruction_str)
)


chat_human_prompt_template = HumanMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["question"] , template="{question}")
)

messages=[chat_system_prompt_template,chat_human_prompt_template]

chat_prompt_template = ChatPromptTemplate(
    input_variables=["context","question"],
    messages=messages
)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

chain = chat_prompt_template | chat_model | StrOutputParser()

result = chain.invoke({
    "context":context,
    "question":question
})

print(result)

Yes, one patient stated, "I had a great stay!"


# **Adding RAG**

In [5]:
! pip install langchain-chroma

In [7]:
import time
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain.document_loaders import CSVLoader

review_csv_path = "/content/reviews.csv"

review_chroma_path = "chroma_data"

loader = CSVLoader(
    file_path = review_csv_path,
    source_column = 'review'
)

reviews = loader.load()

embedding_function = GoogleGenerativeAIEmbeddings(
    model='models/gemini-embedding-001',
    google_api_key = userdata.get("smart_IDP")
)

batch_size = 20

num_batches = (len(reviews)-1) // batch_size+1
reviews_verctor_db = None

for i in range(0,len(reviews),batch_size):
  batch_docs = reviews[i:i+batch_size]
  current_batch = i // batch_size + 1

  print(f"Processing batch {current_batch}/{num_batches}...")
  if i==0:
    reviews_verctor_db = Chroma.from_documents(
        documents=batch_docs,
        embedding=embedding_function,
        persist_directory=review_chroma_path
    )

  else:
    reviews_verctor_db.add_documents(documents=batch_docs)

  print(f"Batch {current_batch} processed. Waiting for 30 seconds...")
  time.sleep(30)

print("Vector database created successfully and saved to the specified directory.")


Processing batch 1/51...
Batch 1 processed. Waiting for 30 seconds...
Processing batch 2/51...
Batch 2 processed. Waiting for 30 seconds...
Processing batch 3/51...
Batch 3 processed. Waiting for 30 seconds...
Processing batch 4/51...
Batch 4 processed. Waiting for 30 seconds...
Processing batch 5/51...
Batch 5 processed. Waiting for 30 seconds...
Processing batch 6/51...
Batch 6 processed. Waiting for 30 seconds...
Processing batch 7/51...
Batch 7 processed. Waiting for 30 seconds...
Processing batch 8/51...
Batch 8 processed. Waiting for 30 seconds...
Processing batch 9/51...
Batch 9 processed. Waiting for 30 seconds...
Processing batch 10/51...
Batch 10 processed. Waiting for 30 seconds...
Processing batch 11/51...
Batch 11 processed. Waiting for 30 seconds...
Processing batch 12/51...
Batch 12 processed. Waiting for 30 seconds...
Processing batch 13/51...
Batch 13 processed. Waiting for 30 seconds...
Processing batch 14/51...
Batch 14 processed. Waiting for 30 seconds...
Processing

# **PUTTING IT ALL TOGETHER**

In [24]:
from langchain_core.prompts import( SystemMessagePromptTemplate , HumanMessagePromptTemplate , ChatPromptTemplate , PromptTemplate)

from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata

from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_chroma import Chroma

from langchain.document_loaders import CSVLoader

instruction_str = """Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information that's not from the context.
If you don't know an answer, say you don't know.

{context}
"""

reviews_system_prompt_template = SystemMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["context"],
    template = instruction_str)
)


reviews_human_prompt_template = HumanMessagePromptTemplate(
    prompt = PromptTemplate(input_variables=["question"],
    template = "{question}")
)


messages = [reviews_system_prompt_template , reviews_human_prompt_template]


reviews_chat_prompt_template = ChatPromptTemplate(
    input_variables=["context","question"],
    messages = messages
)


print(f"Creating Chat Model...")

reviews_chat_model = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature = 0,
    google_api_key = userdata.get('ecommerce_db')
)


print(f"Chat Model Created.")


# reviews_cs_path = "/content/reviews.csv"


# loader = CSVLoader(
#     file_path = reviews_cs_path,
#     source_column = 'review'
# )


# reviews = loader.load()


# reviews_chroma_path = "chroma_data"


# embedding_function = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001",
#     google_api_key=userdata.get("smart_IDP")
# )


# batch_size = 20


# num_of_batches = (len(reviews) - 1 ) // batch_size + 1


# for i in range(0,len(reviews),batch_size):
#   batch_docs = reviews[i:i+batch_size]
#   current_batch = i // batch_size+1

#   print(f"Processing batch {current_batch}/{num_of_batches}...")

#   if i==0:
#     reviews_vector_db = Chroma.from_documents(
#         documents = batch_docs,
#         embedding = embedding_function,
#         persist_directory = reviews_chroma_path
#     )
#   else:
#     reviews_vector_db.add_documents(documents=batch_docs)

#   print(f"Batch {current_batch} processed. Waiting for 30 seconds...")
#   time.sleep(30)

# print("Vector database created successfully and saved to the specified directory")



Creating Chat Model...
Chat Model Created.


In [25]:
from langchain.schema.runnable import RunnablePassthrough  # Allows passing inputs through unchanged in a pipeline
from langchain_core.output_parsers import StrOutputParser

reviews_retriever = reviews_verctor_db.as_retriever(k=10)

review_chain = (
    {"context":reviews_retriever , "question":RunnablePassthrough()} | reviews_chat_prompt_template | reviews_chat_model | StrOutputParser()
)

In [26]:
question = """Has anyone complained about communication with the hospital staff?"""
review_chain.invoke(question)

GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier"
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
]

In [20]:
! pip install gradio

In [22]:
def respond_to_user_question(question:str , history:list) -> str:
  """
  Respond the user's question using the review_chain
  """
  return review_chain.invoke(question)

In [23]:
respond_to_user_question("Has anyone complaied about communication with the hospital staff?",[])

GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerDayPerUserPerProjectPerModel-FreeTier"
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
]

In [ ]:
import gradio as gr
interface = gr.ChatInterface(fn=respond_to_user_question, title="Review Helper Bot")

interface.launch(debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://221670b12cfafe299e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/embeddings.py", line 324, in embed_query
    result: EmbedContentResponse = self.client.embed_content(request)
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1306, in embed_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
            